In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
# %%


# spark-sql --packages org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1\
#     --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions \
#     --conf spark.sql.catalog.spark_catalog=org.apache.iceberg.spark.SparkSessionCatalog \
#     --conf spark.sql.catalog.spark_catalog.type=hive \
#     --conf spark.sql.catalog.local=org.apache.iceberg.spark.SparkCatalog \
#     --conf spark.sql.catalog.local.type=hadoop \
#     --conf spark.sql.catalog.local.warehouse=$PWD/warehouse

# spark-shell --packages org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1


spark_configs = {
    "spark.jars.packages": "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1",
    "spark.sql.extensions": "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.catalog.spark_catalog": "org.apache.iceberg.spark.SparkSessionCatalog",
    "spark.sql.catalog.spark_catalog.type": "hive",
    "spark.sql.catalog.local":"org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.local.warehouse":"./warehouse",
    "spark.sql.catalog.local.type":"hadoop",
}

spark = (
    SparkSession
    .builder
    .appName("Python Spark SQL basic example")
    .config(map=spark_configs)
    .getOrCreate()
)

25/01/20 11:39:57 WARN Utils: Your hostname, codespaces-7465f4 resolves to a loopback address: 127.0.0.1; using 10.0.1.202 instead (on interface eth0)
25/01/20 11:39:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2122c437-a131-4bd6-bdfe-6f3c84979576;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 in central
:: resolution report :: resolve 130ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apa

In [3]:
spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [4]:
spark.sql("CREATE OR REPLACE TABLE local.db.table (id bigint, data string) USING iceberg;")
spark.read.table("local.db.table").show()

+---+----+
| id|data|
+---+----+
+---+----+



In [5]:
spark.sql("INSERT INTO local.db.table VALUES (1, 'a'), (2, 'b'), (3, 'c');")
spark.read.table("local.db.table").show()

+---+----+
| id|data|
+---+----+
|  1|   a|
|  2|   b|
|  3|   c|
+---+----+



In [6]:
spark.range(10).withColumn("data", F.lit("mydata")).select("id", "data").writeTo("local.db.table").append()
spark.read.table("local.db.table").show()


+---+------+
| id|  data|
+---+------+
|  1|     a|
|  2|     b|
|  3|     c|
|  0|mydata|
|  1|mydata|
|  2|mydata|
|  3|mydata|
|  4|mydata|
|  5|mydata|
|  6|mydata|
|  7|mydata|
|  8|mydata|
|  9|mydata|
+---+------+



In [7]:
spark.read.table("local.db.table.snapshots").show()

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2025-01-20 11:17:...|6525315697047048531|               NULL|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:21:...|2719531211674363846|6525315697047048531|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:21:...|7517947901201637501|2719531211674363846|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:39:...|9090593959980706118|               NULL|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:39:...|8399356804968829315|9090593959980706118|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:40:...|3389023457785785759|               NULL|   append|warehous

In [8]:
# Time Travel

(spark.read
    .option("snapshot-id", 6525315697047048531)
    .table("local.db.table")).show()

+---+----+
| id|data|
+---+----+
|  1|   a|
|  2|   b|
|  3|   c|
+---+----+



In [9]:
sql = """
DESCRIBE EXTENDED local.db.table
"""
spark.sql(sql).show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                  id|              bigint|   NULL|
|                data|              string|   NULL|
|                    |                    |       |
|  # Metadata Columns|                    |       |
|            _spec_id|                 int|       |
|          _partition|            struct<>|       |
|               _file|              string|       |
|                _pos|              bigint|       |
|            _deleted|             boolean|       |
|                    |                    |       |
|# Detailed Table ...|                    |       |
|                Name|      local.db.table|       |
|                Type|             MANAGED|       |
|            Location|./warehouse/db/table|       |
|            Provider|             iceberg|       |
|               Owner|           codespace|       |
|    Table P

In [10]:
sql = """
DESCRIBE local.db.table
"""
spark.sql(sql).show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|      id|   bigint|   NULL|
|    data|   string|   NULL|
+--------+---------+-------+



25/01/20 11:40:14 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [12]:
spark.sql("SHOW TBLPROPERTIES local.db.table").show()

+--------------------+------------------+
|                 key|             value|
+--------------------+------------------+
| current-snapshot-id|111972506823236605|
|              format|   iceberg/parquet|
|      format-version|                 2|
|write.parquet.com...|              zstd|
+--------------------+------------------+



In [49]:
spark.read.format("iceberg").load("./warehouse/db/table").show()

+---+------+
| id|  data|
+---+------+
|  0|mydata|
|  1|mydata|
|  2|mydata|
|  3|mydata|
|  4|mydata|
|  5|mydata|
|  6|mydata|
|  7|mydata|
|  8|mydata|
|  9|mydata|
|  1|     a|
|  2|     b|
|  3|     c|
+---+------+



In [53]:
(spark.read
  .format("iceberg")
  .option("start-snapshot-id", "9090593959980706118")
  .option("end-snapshot-id", "8399356804968829315")
  .load("./warehouse/db/table")).show()

+---+----+
| id|data|
+---+----+
|  1|   a|
|  2|   b|
|  3|   c|
+---+----+



In [56]:
spark.read.table("local.db.table.snapshots").show()


+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2025-01-20 11:17:...|6525315697047048531|               NULL|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:21:...|2719531211674363846|6525315697047048531|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:21:...|7517947901201637501|2719531211674363846|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:39:...|9090593959980706118|               NULL|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:39:...|8399356804968829315|9090593959980706118|   append|warehouse/db/tabl...|{spark.app.id -> ...|
|2025-01-20 11:40:...|3389023457785785759|               NULL|   append|warehous

In [55]:
spark.read.table("local.db.table.history").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2025-01-20 11:17:...|6525315697047048531|               NULL|              false|
|2025-01-20 11:21:...|2719531211674363846|6525315697047048531|              false|
|2025-01-20 11:21:...|7517947901201637501|2719531211674363846|              false|
|2025-01-20 11:39:...|9090593959980706118|               NULL|              false|
|2025-01-20 11:39:...|8399356804968829315|9090593959980706118|              false|
|2025-01-20 11:40:...|3389023457785785759|               NULL|               true|
|2025-01-20 11:40:...| 111972506823236605|3389023457785785759|               true|
+--------------------+-------------------+-------------------+-------------------+



In [57]:
spark.read.table("local.db.table.entries").show()


+------+-------------------+---------------+--------------------+--------------------+--------------------+
|status|        snapshot_id|sequence_number|file_sequence_number|           data_file|    readable_metrics|
+------+-------------------+---------------+--------------------+--------------------+--------------------+
|     1| 111972506823236605|              7|                   7|{0, warehouse/db/...|{{72, 5, 0, NULL,...|
|     1| 111972506823236605|              7|                   7|{0, warehouse/db/...|{{72, 5, 0, NULL,...|
|     1|3389023457785785759|              6|                   6|{0, warehouse/db/...|{{37, 1, 0, NULL,...|
|     1|3389023457785785759|              6|                   6|{0, warehouse/db/...|{{42, 2, 0, NULL,...|
+------+-------------------+---------------+--------------------+--------------------+--------------------+



In [60]:
spark.read.table("local.db.table.files").show(truncate=False)


+-------+------------------------------------------------------------------------------------+-----------+-------+------------+------------------+------------------+----------------+-----------------+----------------+----------------------------------------------------------+----------------------------------------------------------+------------+-------------+------------+-------------+----------------------------------------------------------+
|content|file_path                                                                           |file_format|spec_id|record_count|file_size_in_bytes|column_sizes      |value_counts    |null_value_counts|nan_value_counts|lower_bounds                                              |upper_bounds                                              |key_metadata|split_offsets|equality_ids|sort_order_id|readable_metrics                                          |
+-------+------------------------------------------------------------------------------------+--------

In [61]:
spark.read.table("local.db.table.partitions").show(truncate=False)


+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+-----------------------+------------------------+
|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|last_updated_at        |last_updated_snapshot_id|
+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+-----------------------+------------------------+
|13          |4         |2700                         |0                           |0                         |0                           |0                         |2025-01-20 11:40:07.752|111972506823236605      |
+------------+----------+-----------------------------+----------------------------+--------------------------+---------------------

In [62]:
spark.read.table("local.db.table.refs").show(truncate=False)


+----+------+------------------+-----------------------+---------------------+----------------------+
|name|type  |snapshot_id       |max_reference_age_in_ms|min_snapshots_to_keep|max_snapshot_age_in_ms|
+----+------+------------------+-----------------------+---------------------+----------------------+
|main|BRANCH|111972506823236605|NULL                   |NULL                 |NULL                  |
+----+------+------------------+-----------------------+---------------------+----------------------+



In [63]:
spark.read.table("local.db.table").show(truncate=False)


+---+------+
|id |data  |
+---+------+
|0  |mydata|
|1  |mydata|
|2  |mydata|
|3  |mydata|
|4  |mydata|
|5  |mydata|
|6  |mydata|
|7  |mydata|
|8  |mydata|
|9  |mydata|
|1  |a     |
|2  |b     |
|3  |c     |
+---+------+



In [64]:
sql = """
UPDATE local.db.table
SET data = 'hello'
WHERE id=0
"""
spark.sql(sql)

DataFrame[]

In [65]:
spark.read.table("local.db.table").show(truncate=False)

+---+------+
|id |data  |
+---+------+
|0  |hello |
|1  |mydata|
|2  |mydata|
|3  |mydata|
|4  |mydata|
|5  |mydata|
|6  |mydata|
|7  |mydata|
|8  |mydata|
|9  |mydata|
|1  |a     |
|2  |b     |
|3  |c     |
+---+------+



In [71]:
spark.range(10).writeTo("local.db.table2").createOrReplace()

In [72]:
spark.read.table("local.db.table2").show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



ModuleNotFoundError: No module named 'pyarrow'